# K-Nearest Neighbourhood - Instance Based/Lazy Algorithm/Non-Parametric

In [ ]:
### Importing Packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import csv
import random
import math
import operator

%matplotlib inline

## How KNN works

![title](img/knn1.png)

---------------------------------

![title](img/knn2.png)



## How do we choose the factor K?
![title](img/dboundary.png)

------------------------------------

![title](img/verror.png)

### Quiz

##### Suppose the test point is x=1, y=1 what will be ur class when k=3??

![](img/knn_quiz.png)

### Step by Step procedures - KNN

### STEP1 - Data Preparation

In [ ]:
with open('data/iris.data', 'r') as csvfile:
    lines = csv.reader(csvfile)
    for row in lines:
        print(', '.join(row))
csvfile.close()

In [ ]:
def loadDataset(filename, split, trainingSet=[] , testSet=[]):
    with open(filename, 'r') as csvfile:
        lines = csv.reader(csvfile)
        dataset = list(lines)
        for x in range(len(dataset)-1):
            for y in range(4):
                dataset[x][y] = float(dataset[x][y])
                if random.random() < split:
                    trainingSet.append(dataset[x])
                else:
                    testSet.append(dataset[x])
csvfile.close()

In [ ]:
trainingSet=[]
testSet=[]
loadDataset('data/iris.data', 0.70, trainingSet, testSet)
print ('Train: ' + repr(len(trainingSet)))
print ('Test: ' + repr(len(testSet)))

### STEP2 - Similarity Measures

In [ ]:
def euclideanDistance(instance1, instance2, length):
    distance = 0
    for x in range(length):
        distance += pow((instance1[x] - instance2[x]), 2)
    return math.sqrt(distance)

In [ ]:
data1 = [2, 2, 2, 'a']
data2 = [4, 4, 4, 'b']

assert len(data1) == len(data2), 'Length Mismatch ERROR'

distance = euclideanDistance(data1, data2, len(data1)-1)
print ('Distance: ', distance)

### STEP3 - Neighbors

In [ ]:
def getNeighbors(trainingSet, testInstance, k):
    global distances, neighbors
    distances = []
    length = len(testInstance)
    for x in range(len(trainingSet)):
        dist = euclideanDistance(testInstance, trainingSet[x], length)
        distances.append((trainingSet[x], dist))
    distances.sort(key=operator.itemgetter(1))
    neighbors = []
    for x in range(k):
        neighbors.append(distances[x][0])
    return neighbors

In [ ]:
trainSet = [[2, 2, 2, 'a'], [4, 4, 4, 'b']]
testInstance = [5, 5, 5]
neighbors = getNeighbors(trainSet, testInstance, k=1)
print(neighbors)

### STEP4 - Response Voting

In [ ]:
def getResponse(neighbors):
    global classVotes
    classVotes = {}
    for x in range(len(neighbors)):
        response = neighbors[x][-1]
        if response in classVotes:
            classVotes[response] += 1
        else:
            classVotes[response] = 1
    sortedVotes = sorted(classVotes.items(), key=operator.itemgetter(1), reverse=True)
    return sortedVotes[0][0]

In [ ]:
neighbors = [[1,1,1,'a'], [2,2,2,'a'], [3,3,3,'b']]
response = getResponse(neighbors)
print(response)

In [ ]:
classVotes

### STEP5 - Accuracy

In [ ]:
def getAccuracy(testSet, predictions):
    correct = 0
    for x in range(len(testSet)):
        if testSet[x][-1] is predictions[x]:
            correct += 1
    return (correct/float(len(testSet))) * 100.0

In [ ]:
testSet = [[1,1,1,'a'], [2,2,2,'a'], [3,3,3,'b']]
predictions = ['a', 'a', 'a']

accuracy = getAccuracy(testSet, predictions)
print(np.round(accuracy, 0))

### STEP6 - Main Program-Including all the functions

In [ ]:
def loadDataset(filename, split, trainingSet=[] , testSet=[]):
    with open(filename, 'r') as csvfile:
        lines = csv.reader(csvfile)
        dataset = list(lines)
        for x in range(len(dataset)-1):
            for y in range(4):
                dataset[x][y] = float(dataset[x][y])
                if random.random() < split:
                    trainingSet.append(dataset[x])
                else:
                    testSet.append(dataset[x])
    csvfile.close()
                
def euclideanDistance(instance1, instance2, length):
    distance = 0
    for x in range(length):
        distance += pow((instance1[x] - instance2[x]), 2)
    return math.sqrt(distance)
 
def getNeighbors(trainingSet, testInstance, k):
    distances = []
    length = len(testInstance)-1
    for x in range(len(trainingSet)):
        dist = euclideanDistance(testInstance, trainingSet[x], length)
        distances.append((trainingSet[x], dist))
    distances.sort(key=operator.itemgetter(1))
    neighbors = []
    for x in range(k):
        neighbors.append(distances[x][0])
    return neighbors
 
def getResponse(neighbors):
    classVotes = {}
    for x in range(len(neighbors)):
        response = neighbors[x][-1]
        if response in classVotes:
            classVotes[response] += 1
        else:
            classVotes[response] = 1
    sortedVotes = sorted(classVotes.items(), key=operator.itemgetter(1), reverse=True)
    return sortedVotes[0][0]


def getAccuracy(testSet, predictions):
    correct = 0
    for x in range(len(testSet)):
        if testSet[x][-1] == predictions[x]:
            correct += 1
    return (correct/float(len(testSet))) * 100.0
 

In [ ]:
def main():
    # prepare data
    global trainingSet, testSet, neigh
    trainingSet=[]
    testSet=[]
    neigh = []
    split = .70
    loadDataset('data/iris.data', split, trainingSet, testSet)
    print ('Train set: ' + repr(len(trainingSet)))
    print ('Test set: ' + repr(len(testSet)))
    # generate predictions
    predictions=[]
    k = 3
    for x in range(len(testSet)):
        neighbors = getNeighbors(trainingSet, testSet[x], k)
        result = getResponse(neighbors)
        predictions.append(result)
        print('> predicted=' + repr(result) + ', actual=' + repr(testSet[x][-1]))
        neigh.append(neighbors)
    accuracy = getAccuracy(testSet, predictions)
    print('Accuracy: ' + repr(accuracy) + '%')

In [ ]:
main()

##### Using SKLEARN 

In [ ]:
X_train = np.array(trainingSet)[:, :3]
y_train = np.array(trainingSet)[:, -1]
X_test = np.array(testSet)[:, :3]
y_test = np.array(testSet)[:, -1]

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
knn = KNeighborsClassifier(n_neighbors=3)

In [ ]:
knn.fit(X_train, y_train)

In [ ]:
y_train_pred = knn.predict(X_train)
y_test_pred = knn.predict(X_test)

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
print ("Training Accuracy:" , accuracy_score(y_train, y_train_pred))
print("\n")
print ("Test Accuracy:" , accuracy_score(y_test, y_test_pred))